In [ ]:
pip install selenium

In [ ]:
import time
import tempfile
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException
)

In [ ]:
def search_product_price(product_name):
    temp_profile = tempfile.mkdtemp()
    options = Options()
    options.add_argument(f"--user-data-dir={temp_profile}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--remote-debugging-port=9222")  # simular DevTools aberto
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114 Safari/537.36")
    options.add_argument("--auto-open-devtools-for-tabs")

    driver = webdriver.Chrome(options=options)

    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    })

    wait = WebDriverWait(driver, 20)
    driver.get("https://www.pingodoce.pt/")
    time.sleep(3)

    try:
        # Fechar popup
        try:
            popup_close = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.icon-close-popup')))
            popup_close.click()
            print("Popup fechado")
        except:
            print("Popup não apareceu")

        # Aceitar cookies
        try:
            cookie_button = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler')))
            cookie_button.click()
            print("Cookies aceites")
        except:
            print("Barra de cookies não apareceu")

        # Esperar que o input esteja presente
        search_input = wait.until(EC.presence_of_element_located((By.ID, "header-search-bar")))
        print("Input de pesquisa localizado")

        # Forçar foco e eventos JS no input
        driver.execute_script("""
            const input = document.getElementById('header-search-bar');
            input.style.display = 'block';
            input.style.visibility = 'visible';
            input.removeAttribute('disabled');
            input.focus();
            input.click();
            input.dispatchEvent(new MouseEvent('mousedown', { bubbles: true }));
            input.dispatchEvent(new MouseEvent('mouseup', { bubbles: true }));
            input.dispatchEvent(new Event('focus', { bubbles: true }));
            input.dispatchEvent(new Event('click', { bubbles: true }));
        """)
        time.sleep(1)

        # Enviar texto com clique e atraso
        search_input = wait.until(EC.element_to_be_clickable((By.ID, "header-search-bar")))
        search_input.click()
        for char in product_name:
            search_input.send_keys(char)
            time.sleep(0.1)
        search_input.send_keys(Keys.RETURN)

        print("Pesquisa enviada com sucesso")
        time.sleep(2)

        # Scroll inicial para garantir que produtos e botão 'Ver mais' aparecem
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)

        # Função auxiliar para clicar no botão 'Ver mais'
        def clicar_em_ver_mais(driver):
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5)

                ver_mais_button = driver.find_element(By.CSS_SELECTOR, "button.more")
                if ver_mais_button.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ver_mais_button)
                    time.sleep(0.5)
                    try:
                        ver_mais_button.click()
                        print("Clique em 'Ver mais'")
                    except ElementClickInterceptedException:
                        driver.execute_script("arguments[0].click();", ver_mais_button)
                        print("Clique forçado com JavaScript")
                    return True
            except NoSuchElementException:
                print("Botão 'Ver mais' não encontrado")
            return False

        # Repetir o clique em "Ver mais" até não haver mais botão
        max_tentativas = 10
        tentativa = 0
        while tentativa < max_tentativas:
            sucesso = clicar_em_ver_mais(driver)
            if not sucesso:
                print("Botão 'Ver mais' não encontrado ou já não existe")
                break
            time.sleep(2)
            tentativa += 1
     

        # Extrair resultados
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.product-name-link')))
        time.sleep(2)

        all_products = driver.find_elements(By.CSS_SELECTOR, 'div.product-name-link')
        visible_products = []
        for p in all_products:
            try:
                if p.is_displayed():
                    visible_products.append(p)
            except StaleElementReferenceException:
                continue

        print(f"{len(visible_products)} produtos visíveis encontrados")

        results = []
        for product in visible_products:
            try:
        # Subir ao elemento pai do produto
                product_tile = product.find_element(By.XPATH, "./ancestor::div[contains(@class, 'product-tile')]")

        # Título
                title = product.find_element(By.CSS_SELECTOR, 'a').text.strip()

        # Preço dentro do pai
                try:
                    price = product_tile.find_element(By.CSS_SELECTOR, 'div.product-price span.sales').text.strip()
                except:
                    price = "Preço não encontrado"

                results.append((title, price))
            except Exception as e:
                print("Erro ao extrair produto:", e)
                continue

        return results

    finally:
        driver.quit()
        shutil.rmtree(temp_profile)

# --- Uso ---
product_name = input("Produto a procurar: ")
prices = search_product_price(product_name)

if prices:
    for title, price in prices:
        print(f"{title} - {price}")
else:
    print("Nenhum produto encontrado.")
